In [35]:
#!/usr/bin/env python
# coding: utf-8

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm

# keras : librairie de deep learning
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, Dense, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [4]:
def lecture(path = None, shape = (500,500), proba_train = 0.2, proba_test = .9, color = 'grey'):
    data_train = []
    data_test = []
    if color == 'grey':
        for i in tqdm(os.listdir(path)):
            if i[-3:] == 'png':
                rand = np.random.random()
                if rand <= proba_train:
                    img = Image.open(path + i)
                    data_train.append(np.mean(np.array(img.resize(shape, Image.BILINEAR)), axis=2))

                elif rand >= proba_test:
                    img = Image.open(path + i)
                    data_test.append(np.mean(np.array(img.resize(shape, Image.BILINEAR)), axis=2))
    
    elif color == 'color':
        for i in tqdm(os.listdir(path)):
            if i[-3:] == 'png':
                rand = np.random.random()
                if rand <= proba_train:
                    img = Image.open(path + i)
                    data_train.append(np.array(img.resize(shape, Image.BILINEAR)))

                elif rand >= proba_test:
                    img = Image.open(path + i)
                    data_test.append(np.array(img.resize(shape, Image.BILINEAR)))
    
            
    return np.array(data_train), np.array(data_test)


In [5]:
A320_train, A320_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Airbus/A320/", (256,256),color='color')


In [6]:
A320_train.shape


(195, 256, 256, 3)

In [7]:
A320_test.shape


(99, 256, 256, 3)

In [8]:
A330_train, A330_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Airbus/A330/", (256,256),color='color')


In [9]:
A350_train, A350_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Airbus/A350/", (256,256),color='color')


In [10]:
A380_train, A380_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Airbus/A380/", (256,256),color='color')


In [11]:
Boeing737_train, Boeing737_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Boeing/737/", (256,256),color='color')


In [12]:
Boeing747_train, Boeing747_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Boeing/747/", (256,256),color='color')


In [13]:
Boeing767_train, Boeing767_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Boeing/767/", (256,256),color='color')


In [14]:
Boeing777_train, Boeing777_test = lecture("/Users/vincent/Desktop/projet/Interpromo2020/All Data/ANALYSE IMAGE/G7_scrapping/Airliners/data/Boeing/777/", (256,256),color='color')


In [15]:
yBoeing_train  = np.repeat('Boeing', len(Boeing777_train)+len(Boeing767_train)+ len(Boeing747_train) + len(Boeing737_train))


In [16]:
yAirbus_train = np.repeat('Airbus', len(A380_train) + len(A350_train) + len(A330_train)+ len(A320_train))


In [17]:
yBoeing_test  = np.repeat('Boeing', len(Boeing777_test)+len(Boeing767_test)+ len(Boeing747_test) + len(Boeing737_test))


In [18]:
yAirbus_test = np.repeat('Airbus', len(A380_test) + len(A350_test) + len(A330_test)+ len(A320_test))



In [19]:
ytrain = np.concatenate((yAirbus_train, yBoeing_train))


In [20]:
ytrain = pd.get_dummies(pd.Series(ytrain)).values


In [21]:
ytest = np.concatenate((yAirbus_test, yBoeing_test))


In [22]:
ytest = pd.get_dummies(pd.Series(ytest)).values


In [23]:
data_train = np.concatenate((A320_train,A330_train,A350_train,A380_train,Boeing777_train,Boeing767_train,Boeing747_train,Boeing737_train))


In [24]:
data_test = np.concatenate((A320_test,A330_test,A350_test,A380_test,Boeing777_test,Boeing767_test,Boeing747_test,Boeing737_test))


In [112]:
#data_train = np.reshape(data_train, (data_train.shape[0], 256, 256, 1))


In [100]:
#data_test = np.reshape(data_test, (data_test.shape[0], 256, 256, 1))

In [25]:
ytrain.shape

(1614, 2)

In [26]:
ytest.shape


(829, 2)

In [27]:
arr = np.arange(ytrain.shape[0])
np.random.shuffle(arr)

ytrain = ytrain[arr]
data_train = data_train[arr]

In [28]:
arr = np.arange(ytest.shape[0])
np.random.shuffle(arr)

ytest = ytest[arr]
data_test = data_train[arr]


In [29]:
def create_model(input_shape = (256,256,1), nb_couches = 5, nb_neuronnes = 10, kernel = (3, 3), pool = (2, 2), nb_classes = ytrain.shape[1]):
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    for i in range(nb_couches):
        model.add(Conv2D(2**(i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
                  
        model.add(Conv2D(2**(i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(MaxPooling2D(pool_size=pool))

    model.add(Flatten())

    model.add(Dense(nb_classes, activation='softmax'))

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model

In [30]:
model = create_model(nb_neuronnes = 6)
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 1)       10        
_________________________________________________________________
batch_normalization_1 (Batch (None, 254, 254, 1)       4         
_________________________________________________________________
re_lu_1 (ReLU)               (None, 254, 254, 1)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 1)       10        
_________________________________________________________________
batch_normalization_2 (Batch (None, 252, 252, 1)       4         
_________________________________________________________________
re_lu_2 (ReLU)               (None, 252, 252, 1)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 1)      

In [32]:
batch_size = 32
epochs = 5

# Lancement de l'entraînement
model.fit(data_train, ytrain, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(data_test, ytest))


Train on 1600 samples, validate on 825 samples
Epoch 1/5
 160/1600 [==>...........................] - ETA: 24:48 - loss: 0.9918 - accuracy: 0.5125

KeyboardInterrupt: 

# Lecture from keras

# RESNET

In [31]:
from keras.applications.resnet import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


In [43]:
# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = (256, 256, 3))

# add a global spatial average pooling layer
x = base_model.output
x = MaxPooling2D((2, 2))(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)
#x = MaxPooling2D(pool_size = (2,2))(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [44]:
batch_size = 32
epochs = 5

# Lancement de l'entraînement
model.fit(data_train, ytrain[:, 0], batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(data_test, ytest[:, 0]))


Train on 1614 samples, validate on 829 samples
Epoch 1/5
1614/1614 [==============================] - 611s 379ms/step - loss: 0.8128 - accuracy: 0.6326 - val_loss: 3.2875 - val_accuracy: 0.5211
Epoch 2/5
1614/1614 [==============================] - 524s 324ms/step - loss: 0.3437 - accuracy: 0.8550 - val_loss: 7.0108 - val_accuracy: 0.5127
Epoch 3/5
  64/1614 [>.............................] - ETA: 5:00 - loss: 0.2808 - accuracy: 0.9219

KeyboardInterrupt: 